In [ ]:
import copy
import csv
import glob
import importlib
import itertools
import operator
import os
import random
import time
from math import exp, log, pi, sqrt
from os.path import *

import keras.layers as layers
import keras.models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import np_utils
from numba import njit
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

import cnn_builder as cbuild
import cnn_runner as crun
import config
import ddpg.learning_nets as ln
import ddpg.task_nets as tn
import dr_methods as drm
import feature_interpretation as cnna
import seg_methods as sm
import niftiutils.helper_fxns as hf
import niftiutils.transforms as tr
import voi_methods as vm

%matplotlib inline
np.set_printoptions(4)

In [174]:
importlib.reload(tn)
M = tn.get_trainable_model()
M.compile(optimizer='adam', loss=None)

In [ ]:
M.summary(120)

In [175]:
importlib.reload(cbuild)
gen = cbuild._train_gen_unet()

In [ ]:
hist = M.fit_generator(gen, steps_per_epoch=10, epochs=10)

[np.exp(K.get_value(log_var[0]))**0.5 for log_var in M.layers[-1].log_vars]

prediction_model = get_prediction_model()
trainable_model = get_trainable_model(prediction_model)
trainable_model.compile(optimizer='adam', loss=None)
assert len(trainable_model.layers[-1].trainable_weights) == 2  # two log_vars, one for each output
assert len(trainable_model.losses) == 1
hist = trainable_model.fit([X, Y1, Y2], nb_epoch=nb_epoch, batch_size=batch_size, verbose=0)

[np.exp(K.get_value(log_var[0]))**0.5 for log_var in trainable_model.layers[-1].log_vars]

In [78]:
importlib.reload(config)
importlib.reload(hf)
importlib.reload(cbuild)
importlib.reload(crun)
C = config.Config('etiology')
T = config.Hyperparams()
T.get_best_hyperparams()
T.epochs = 30
T.steps_per_epoch = 300

In [ ]:
importlib.reload(cbuild)
importlib.reload(dqna)
#dqn_generator = cbuild._train_gen_dqn([])
agent = dqna.train_dqn(dqn_generator, agent)

In [ ]:
agent.cls_model.fit(cls_generator)

In [9]:
T.padding = ['same', 'same']
T.pool_sizes = [(2,2,2),(2,2,2)]
T.f = [64,64,64,64,64,64,64]
T.skip_con = True
T.epochs = 20

In [ ]:
drm.dcm2npy_batch(acc_nums=["E100113043"])
#vm.reset_accnum('E105464882')

In [ ]:
vm.plot_check(2, "E102088195");

In [ ]:
vm.xref_dirs_with_excel()

In [ ]:
crun.run_fixed_hyperparams([C], hyperparams=T)#C_list)

In [ ]:
importlib.reload(cbuild)
model = cbuild.build_cnn_hyperparams(T)
model.summary(line_length=120)

In [ ]:
importlib.reload(cbuild)
#Z_reader = ['E103312835_1','12823036_0','12569915_0','E102093118_0','E102782525_0','12799652_0','E100894274_0','12874178_3','E100314676_0','12842070_0','13092836_2','12239783_0','12783467_0','13092966_0','E100962970_0','E100183257_1','E102634440_0','E106182827_0','12582632_0','E100121654_0','E100407633_0','E105310461_0','12788616_0','E101225606_0','12678910_1','E101083458_1','12324408_0','13031955_0','E101415263_0','E103192914_0','12888679_2','E106096969_0','E100192709_1','13112385_1','E100718398_0','12207268_0','E105244287_0','E102095465_0','E102613189_0','12961059_0','11907521_0','E105311123_0','12552705_0','E100610622_0','12975280_0','E105918926_0','E103020139_1','E101069048_1','E105427046_0','13028374_0','E100262351_0','12302576_0','12451831_0','E102929168_0','E100383453_0','E105344747_0','12569826_0','E100168661_0','12530153_0','E104697262_0']
X_test, Y_test, train_generator, num_samples, train_orig, Z = cbuild.get_cnn_data(n=4)#, Z_test_fixed=Z_reader)
Z_test, Z_train_orig = Z
X_train_orig, Y_train_orig = train_orig
hist = model.fit_generator(train_generator, steps_per_epoch=T.steps_per_epoch, epochs=T.epochs, validation_data=[X_test, Y_test])#, callbacks=[T.early_stopping])

In [10]:
model.save(join(C.model_dir, "model_.hdf5"))

In [ ]:
model.predict(X_train_orig[20:40])

In [ ]:
Y_train_orig[:20]

In [ ]:
hist = model.fit_generator(train_generator, steps_per_epoch=T.steps_per_epoch, epochs=T.epochs, validation_data=[X_test, Y_test])#, callbacks=[T.early_stopping])

In [ ]:
#plot_with_bbox(fn_list[2], cls_mapping[wrong_guesses[2]])
Y_pred = model.predict(X_test)
y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_test])
y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])

cm = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")

#save_output(Z_test, y_pred, y_true)

print(accuracy_score(y_true, y_pred))
#y_true_simp, y_pred_simp, _ = cnna.merge_classes(y_true, y_pred)
#print(accuracy_score(y_true_simp, y_pred_simp))